<a href="https://colab.research.google.com/github/starlingomez/preworkmac/blob/master/30_DE_NOVIEMBRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#cody martin

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [5.6], # Changed to list
    'avg_min_last5': [26.9], # Changed to list
    'opp_avg_reb_allowed': [45.2], # Changed to list
    'player_avg_reb_vs_opp': [5], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 83.5233 - mae: 8.6779 - val_loss: 76.1521 - val_mae: 8.3509
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 64.1480 - mae: 7.5175 - val_loss: 51.3347 - val_mae: 6.7218
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 39.2007 - mae: 5.6667 - val_loss: 25.0083 - val_mae: 4.5270
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 17.0020 - mae: 3.4677 - val_loss: 8.9245 - val_mae: 2.5688
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 6.7796 - mae: 2.1040 - val_loss: 5.2142 - val_mae: 1.8548
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 5.4128 - mae: 1.8553 - val_loss: 4.8458 - val_mae: 1.7672
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.2245 - mae: 1.8284 - val_loss: 4.7566 - val_mae: 1.7467
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.9705 - mae: 1.7910 - val_loss: 4.7221 - val_mae: 1.7429
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - l

#josh green

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [2.8], # Changed to list
    'avg_min_last5': [30.1], # Changed to list
    'opp_avg_reb_allowed': [45.2], # Changed to list
    'player_avg_reb_vs_opp': [3], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 77.1760 - mae: 8.3006 - val_loss: 72.5926 - val_mae: 8.1002
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 58.7794 - mae: 7.0849 - val_loss: 47.3990 - val_mae: 6.3808
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 35.3494 - mae: 5.2775 - val_loss: 21.6760 - val_mae: 4.0964
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 14.0730 - mae: 3.1157 - val_loss: 7.4072 - val_mae: 2.1995
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.0313 - mae: 2.1379 - val_loss: 4.5474 - val_mae: 1.6278
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.3461 - mae: 1.8537 - val_loss: 4.3450 - val_mae: 1.5769
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.5563 - mae: 1.8607 - val_loss: 4.3239 - val_mae: 1.5700
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.3156 - mae: 1.8254 - val_loss: 4.2390 - val_mae: 1.5464
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.9798 

#lamelo ball

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [6.4], # Changed to list
    'avg_min_last5': [34.3], # Changed to list
    'opp_avg_reb_allowed': [45.2], # Changed to list
    'player_avg_reb_vs_opp': [5], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 84.8239 - mae: 8.7471 - val_loss: 74.4474 - val_mae: 8.2172
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 61.8669 - mae: 7.3135 - val_loss: 47.2570 - val_mae: 6.3206
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.0490 - mae: 5.3494 - val_loss: 20.6128 - val_mae: 3.9619
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.1078 - mae: 3.0757 - val_loss: 7.2506 - val_mae: 2.2295
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.3330 - mae: 2.1523 - val_loss: 4.6760 - val_mae: 1.7077
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.4202 - mae: 1.8363 - val_loss: 4.5338 - val_mae: 1.6717
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.9250 - mae: 1.7611 - val_loss: 4.4544 - val_mae: 1.6418
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.4114 - mae: 1.8445 - val_loss: 4.4544 - val_mae: 1.6455
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.7522 - 

#clint capela

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [9.6], # Changed to list
    'avg_min_last5': [23.1], # Changed to list
    'opp_avg_reb_allowed': [43.2], # Changed to list
    'player_avg_reb_vs_opp': [7], # Changed to list
    'home_indicator': [0], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 80.7780 - mae: 8.5306 - val_loss: 75.5175 - val_mae: 8.2810
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 62.1927 - mae: 7.3269 - val_loss: 49.0390 - val_mae: 6.4910
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.7708 - mae: 5.4674 - val_loss: 22.2008 - val_mae: 4.1558
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17.1020 - mae: 3.4127 - val_loss: 8.1568 - val_mae: 2.3863
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.9906 - mae: 2.0997 - val_loss: 5.1566 - val_mae: 1.8307
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.9182 - mae: 1.9236 - val_loss: 4.6098 - val_mae: 1.6735
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.8160 - mae: 1.7199 - val_loss: 4.5804 - val_mae: 1.6528
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.4484 - mae: 1.8823 - val_loss: 4.4607 - val_mae: 1.6110
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.6109 - 

#okongu

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [6.4], # Changed to list
    'avg_min_last5': [23.5], # Changed to list
    'opp_avg_reb_allowed': [43.2], # Changed to list
    'player_avg_reb_vs_opp': [8], # Changed to list
    'home_indicator': [0], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 89.0675 - mae: 8.9940 - val_loss: 77.7772 - val_mae: 8.4317
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 66.1518 - mae: 7.6221 - val_loss: 55.7184 - val_mae: 6.9919
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.6931 - mae: 5.9093 - val_loss: 29.8690 - val_mae: 4.9230
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 20.3789 - mae: 3.8111 - val_loss: 10.9738 - val_mae: 2.7145
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.6750 - mae: 2.3217 - val_loss: 5.4169 - val_mae: 1.7990
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8349 - mae: 1.8976 - val_loss: 4.6230 - val_mae: 1.6670
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6218 - mae: 1.8986 - val_loss: 4.4525 - val_mae: 1.6419
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.2832 - mae: 1.8210 - val_loss: 4.4054 - val_mae: 1.6371
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.3489 -

#jalen duren

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [8.8], # Changed to list
    'avg_min_last5': [24], # Changed to list
    'opp_avg_reb_allowed': [44.9], # Changed to list
    'player_avg_reb_vs_opp': [6], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 87.2557 - mae: 8.8596 - val_loss: 80.9611 - val_mae: 8.5733
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 66.9110 - mae: 7.6354 - val_loss: 59.0791 - val_mae: 7.1582
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 46.1286 - mae: 6.0202 - val_loss: 34.1053 - val_mae: 5.3003
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26.9994 - mae: 4.3800 - val_loss: 15.2603 - val_mae: 3.4250
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.3601 - mae: 2.7158 - val_loss: 7.3294 - val_mae: 2.1886
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.9458 - mae: 2.1129 - val_loss: 5.4958 - val_mae: 1.8440
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7293 - mae: 1.9062 - val_loss: 4.9429 - val_mae: 1.7159
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0798 - mae: 1.7892 - val_loss: 4.8188 - val_mae: 1.6958
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3366 

#tobias harris

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [5.3], # Changed to list
    'avg_min_last5': [31.8], # Changed to list
    'opp_avg_reb_allowed': [44.9], # Changed to list
    'player_avg_reb_vs_opp': [14], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 85.5978 - mae: 8.7554 - val_loss: 77.8507 - val_mae: 8.4237
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 64.9178 - mae: 7.5043 - val_loss: 53.8703 - val_mae: 6.8670
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.2807 - mae: 5.7572 - val_loss: 27.2864 - val_mae: 4.6788
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.7875 - mae: 3.7573 - val_loss: 9.1247 - val_mae: 2.5204
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.5570 - mae: 2.2190 - val_loss: 4.5154 - val_mae: 1.6811
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6470 - mae: 1.8789 - val_loss: 4.1566 - val_mae: 1.5788
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.2376 - mae: 1.8171 - val_loss: 4.2298 - val_mae: 1.5969
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.2431 - mae: 1.7905 - val_loss: 4.1907 - val_mae: 1.5861
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - los

#cade Cunnunhan

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [6.5], # Changed to list
    'avg_min_last5': [34.9], # Changed to list
    'opp_avg_reb_allowed': [44.9], # Changed to list
    'player_avg_reb_vs_opp': [5], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [1], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 83.5778 - mae: 8.6099 - val_loss: 76.9018 - val_mae: 8.3534
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 61.1782 - mae: 7.2849 - val_loss: 53.2815 - val_mae: 6.8090
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.3944 - mae: 5.6955 - val_loss: 26.1373 - val_mae: 4.5092
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 17.6317 - mae: 3.5837 - val_loss: 8.7148 - val_mae: 2.4231
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.7040 - mae: 2.2353 - val_loss: 4.6933 - val_mae: 1.7126
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6614 - mae: 1.9074 - val_loss: 4.1765 - val_mae: 1.5958
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3995 - mae: 1.8661 - val_loss: 4.1713 - val_mae: 1.6054
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.7261 - mae: 1.7408 - val_loss: 4.1359 - val_mae: 1.5960
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - los

#giannis ante

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [10.8], # Changed to list
    'avg_min_last5': [34.3], # Changed to list
    'opp_avg_reb_allowed': [48.3], # Changed to list
    'player_avg_reb_vs_opp': [12.3], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 86.5546 - mae: 8.8840 - val_loss: 83.4389 - val_mae: 8.7638
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 68.3739 - mae: 7.7785 - val_loss: 64.7070 - val_mae: 7.6405
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 50.0453 - mae: 6.5389 - val_loss: 41.1645 - val_mae: 5.9396
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.6377 - mae: 4.7216 - val_loss: 18.3301 - val_mae: 3.7442
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 12.6249 - mae: 2.8594 - val_loss: 6.5333 - val_mae: 2.1248
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.0992 - mae: 2.0022 - val_loss: 4.3254 - val_mae: 1.6405
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3678 - mae: 1.8781 - val_loss: 4.2039 - val_mae: 1.6206
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.9617 - mae: 1.8088 - val_loss: 4.2437 - val_mae: 1.6333
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.2307 

#alexandre Sarr

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [7.8], # Changed to list
    'avg_min_last5': [27.5], # Changed to list
    'opp_avg_reb_allowed': [45.5], # Changed to list
    'player_avg_reb_vs_opp': [0], # Changed to list
    'home_indicator': [0], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 74.8924 - mae: 8.0929 - val_loss: 66.1414 - val_mae: 7.6132
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 52.5324 - mae: 6.5456 - val_loss: 39.8968 - val_mae: 5.6370
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.6175 - mae: 4.6157 - val_loss: 17.3766 - val_mae: 3.6462
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13.1803 - mae: 2.9571 - val_loss: 7.6241 - val_mae: 2.2219
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 8.1781 - mae: 2.2678 - val_loss: 5.2622 - val_mae: 1.8050
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8200 - mae: 1.9578 - val_loss: 4.9168 - val_mae: 1.7259
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.7100 - mae: 1.9055 - val_loss: 4.7572 - val_mae: 1.6953
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.8500 - mae: 1.7801 - val_loss: 4.6717 - val_mae: 1.6738
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - los

#j valaciunas

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [7.2], # Changed to list
    'avg_min_last5': [17.1], # Changed to list
    'opp_avg_reb_allowed': [45.5], # Changed to list
    'player_avg_reb_vs_opp': [11.5], # Changed to list
    'home_indicator': [0], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 79.8118 - mae: 8.4874 - val_loss: 75.1283 - val_mae: 8.2969
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 59.2016 - mae: 7.2069 - val_loss: 51.8878 - val_mae: 6.7783
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 39.7509 - mae: 5.7583 - val_loss: 24.9361 - val_mae: 4.5125
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 17.1272 - mae: 3.5489 - val_loss: 8.2862 - val_mae: 2.4722
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 6.7643 - mae: 2.0682 - val_loss: 5.1910 - val_mae: 1.8098
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.4442 - mae: 1.8629 - val_loss: 4.9008 - val_mae: 1.7374
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7653 - mae: 1.9188 - val_loss: 4.8507 - val_mae: 1.7338
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0785 - mae: 1.8105 - val_loss: 4.7524 - val_mae: 1.7165
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.6975 - 

#jussuf nurki

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [9.5], # Changed to list
    'avg_min_last5': [22.7], # Changed to list
    'opp_avg_reb_allowed': [46.6], # Changed to list
    'player_avg_reb_vs_opp': [9.5], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 78.2273 - mae: 8.3656 - val_loss: 71.8095 - val_mae: 8.0661
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 56.9686 - mae: 7.0401 - val_loss: 46.0260 - val_mae: 6.2892
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 31.5043 - mae: 4.9566 - val_loss: 19.8352 - val_mae: 3.8619
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.1995 - mae: 2.8813 - val_loss: 6.6094 - val_mae: 2.1563
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.8252 - mae: 2.1308 - val_loss: 4.5767 - val_mae: 1.6998
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.5730 - mae: 1.9248 - val_loss: 4.4138 - val_mae: 1.6671
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.2113 - mae: 1.8145 - val_loss: 4.3794 - val_mae: 1.6613
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.1993 - mae: 1.8384 - val_loss: 4.2937 - val_mae: 1.6319
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0312 - 

#DEVIN BOOKER

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [3.2], # Changed to list
    'avg_min_last5': [37.7], # Changed to list
    'opp_avg_reb_allowed': [46.6], # Changed to list
    'player_avg_reb_vs_opp': [5.5], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 83.2189 - mae: 8.6368 - val_loss: 76.4748 - val_mae: 8.3364
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 63.8955 - mae: 7.4402 - val_loss: 52.0804 - val_mae: 6.6713
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.8252 - mae: 5.7225 - val_loss: 25.7285 - val_mae: 4.3887
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15.7011 - mae: 3.2420 - val_loss: 10.1572 - val_mae: 2.7028
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.6885 - mae: 2.2523 - val_loss: 6.0130 - val_mae: 2.0043
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.6051 - mae: 1.9083 - val_loss: 5.2554 - val_mae: 1.8427
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6259 - mae: 1.9063 - val_loss: 5.0086 - val_mae: 1.7831
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0807 - mae: 1.8009 - val_loss: 4.8642 - val_mae: 1.7385
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.0995 

#jonh Collins

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Set a random seed for reproducibility
np.random.seed(42)

# Number of samples in the synthetic dataset
num_samples = 1000

# Generate synthetic features
avg_reb_last5 = np.random.uniform(3, 15, num_samples)         # Player's avg rebounds in last 5 games
avg_min_last5 = np.random.uniform(10, 40, num_samples)        # Player's avg minutes in last 5 games
opp_avg_reb_allowed = np.random.uniform(40, 60, num_samples)  # Opponent's avg rebounds allowed
player_avg_reb_vs_opp = np.random.uniform(3, 15, num_samples) # Player's avg rebounds vs opponent
home_indicator = np.random.randint(0, 2, num_samples)         # 1 if home game, 0 if away
back_to_back = np.random.randint(0, 2, num_samples)           # 1 if back-to-back game, 0 otherwise

# Create a synthetic target variable (rebounds in the next game)
rebounds_next_game = (
    0.5 * avg_reb_last5 +
    0.3 * (avg_min_last5 / 40) * 15 +  # Scale minutes to rebounds
    0.2 * player_avg_reb_vs_opp +
    np.random.normal(0, 2, num_samples)  # Add some noise
)

# Ensure rebounds are within a realistic range
rebounds_next_game = np.clip(rebounds_next_game, 0, 20)

# Assemble the dataset into a DataFrame
data = pd.DataFrame({
    'avg_reb_last5': avg_reb_last5,
    'avg_min_last5': avg_min_last5,
    'opp_avg_reb_allowed': opp_avg_reb_allowed,
    'player_avg_reb_vs_opp': player_avg_reb_vs_opp,
    'home_indicator': home_indicator,
    'back_to_back': back_to_back,
    'rebounds_next_game': rebounds_next_game
})

# Separate features and target variable
X = data.drop('rebounds_next_game', axis=1)
y = data['rebounds_next_game']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Feature scaling for better neural network performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # Output layer for regression
])

# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model on the training data
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model's performance on the test data
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE (Mean Absolute Error): {mae:.2f}")

# Predict rebounds for the next game using the test set
predictions = model.predict(X_test_scaled)

# Display the first 10 predictions alongside actual values
results = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test.values})
print(results.head(10))

# ... (previous code) ...

# Now, let's predict the total rebounds for a specific player's next game

# Define the player's stats
player_data = {
    'avg_reb_last5': [11.2], # Changed to list
    'avg_min_last5': [31.7], # Changed to list
    'opp_avg_reb_allowed': [43.9], # Changed to list
    'player_avg_reb_vs_opp': [8], # Changed to list
    'home_indicator': [1], # Changed to list
    'back_to_back': [0], # Changed to list
}

player_df = pd.DataFrame(player_data)  # Now this should work correctly

# ... (rest of the code) ...

# Scale the player's features using the same scaler
player_scaled = scaler.transform(player_df)

# Predict the rebounds for the player's next game
player_prediction = model.predict(player_scaled)

# Output the predicted total rebounds
print(f"Predicted total rebounds for the player's next game: {player_prediction[0][0]:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 83.7911 - mae: 8.6819 - val_loss: 73.7025 - val_mae: 8.1881
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 58.3048 - mae: 7.1136 - val_loss: 49.2552 - val_mae: 6.5391
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 37.8223 - mae: 5.5068 - val_loss: 22.9193 - val_mae: 4.1953
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 15.2585 - mae: 3.2590 - val_loss: 8.1046 - val_mae: 2.3608
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.5030 - mae: 2.0653 - val_loss: 5.0451 - val_mae: 1.8000
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 5.8971 - mae: 1.9278 - val_loss: 4.6864 - val_mae: 1.6971
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 5.2942 - mae: 1.8513 - val_loss: 4.6084 - val_mae: 1.6668
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 4.9521 - mae: 1.7821 - val_loss: 4.5302 - val_mae: 1.6508
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step